In [ ]:
'''
Some of the exoarchive systems have missing values.  Luckily, the Gaia Data Release 2 can fill some of those values.
Steps:
1) extracts the exoarchive data
2) removes any planets whose detection methods were not transit, RV, or TTV and any planets that are controversially
flagged
3) if one planet in a system has a measured value that is the same for every planet in the system (i.e. stellar mass/radius),
those values are duplicated for every other planet
4) crossmatch appropriate values
5) write out the final crossmatched catalog
'''

In [1]:
# IMPORT STATEMENTS AND PRELIMINARIES

import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy import stats

from astropy.constants import R_earth
from astropy.constants import R_jup

RERJ = float(R_earth/R_jup)


pi = np.pi

MAINPATH = '/Users/research/projects/oviraptor/'

# Read in exoarchive data

In [2]:
# file names
planet_file = MAINPATH + "catalogues/exoarchive_ipac_confirmed_20201008.csv"

# convenience function to read in csv file
def read_csv_file(filename):
    data = []
    with open(filename) as infile:
        reader = csv.reader(infile)

        for row in reader:
            data.append(row)

        keys   = data[97]
        values = data[98:]
            
        return keys, values


# READ IN DR25 DATABASE -- https://exoplanetarchive.ipac.caltech.edu
csv_keys, csv_data = read_csv_file(planet_file)

In [3]:
# convenience functions to pull data from csv files
def getdata(keyname,keys=csv_keys,data=csv_data):
    '''
    keyname = (string) of column definition, see CKS documentation
    '''
    kid = keys.index(keyname)
    
    outdata = []
    for row in data:
        outdata.append(row[kid])
    
    return outdata

In [4]:
# read data into a dictionary
data = {}
for k in csv_keys:
    data[k] = getdata(k)

In [5]:
def check_lengths(data):
    keys = data.keys()
    k0   = list(keys)[0]
    L0   = len(data[k0])
    
    for k in keys:
        if len(data[k]) != L0:
            raise ValueError('inconsistent array lengths')
            
    return None


def convert_to_arrays(data):
    keys = data.keys()
    dnew = {}
    
    for k in keys:
        dnew[k] = np.asarray(data[k])
        
    return dnew       



# grab a reference key
k0 = list(data.keys())[0]


# convert to arrays
data = convert_to_arrays(data)
print('total number of queried objects =', len(data[k0]))


check_lengths(data)

total number of queried objects = 4292


# Remove unwanted objects

In [6]:
# filter detection methods
keep = (data["discoverymethod"] == "Transit") + (data["discoverymethod"] == "Radial Velocity") + \
       (data["discoverymethod"] == "Transit Timing Variations")

for k in data.keys():
    data[k] = data[k][keep]

print("removed {0} objects due to non-relevant DETECTION METHOD".format(np.sum(~keep)))


# controversial flag
bad = data["pl_controv_flag"] == "1"

for k in data.keys():
    data[k] = data[k][~bad]

print("removed {0} objects flagged as CONTROVERSIAL".format(np.sum(bad)))



print("\nafter cuts, {0} objects remain\n".format(len(data[k0])))

print("{0} TRANSITING planets".format(np.sum(data["discoverymethod"] == "Transit")))
print("{0} RADIAL VELOCITY planets".format(np.sum(data["discoverymethod"] == "Radial Velocity")))
print("{0} TTV planets".format(np.sum(data["discoverymethod"] == "Transit Timing Variations")))

removed 185 objects due to non-relevant DETECTION METHOD
removed 10 objects flagged as CONTROVERSIAL

after cuts, 4097 objects remain

3264 TRANSITING planets
812 RADIAL VELOCITY planets
21 TTV planets


In [7]:
data.keys()

dict_keys(['pl_name', 'hostname', 'default_flag', 'sy_snum', 'sy_pnum', 'discoverymethod', 'disc_year', 'disc_facility', 'soltype', 'pl_controv_flag', 'pl_refname', 'pl_orbper', 'pl_orbpererr1', 'pl_orbpererr2', 'pl_orbperlim', 'pl_orbsmax', 'pl_orbsmaxerr1', 'pl_orbsmaxerr2', 'pl_orbsmaxlim', 'pl_rade', 'pl_radeerr1', 'pl_radeerr2', 'pl_radelim', 'pl_radj', 'pl_radjerr1', 'pl_radjerr2', 'pl_radjlim', 'pl_bmasse', 'pl_bmasseerr1', 'pl_bmasseerr2', 'pl_bmasselim', 'pl_bmassj', 'pl_bmassjerr1', 'pl_bmassjerr2', 'pl_bmassjlim', 'pl_bmassprov', 'pl_orbeccen', 'pl_orbeccenerr1', 'pl_orbeccenerr2', 'pl_orbeccenlim', 'pl_insol', 'pl_insolerr1', 'pl_insolerr2', 'pl_insollim', 'pl_eqt', 'pl_eqterr1', 'pl_eqterr2', 'pl_eqtlim', 'ttv_flag', 'st_refname', 'st_teff', 'st_tefferr1', 'st_tefferr2', 'st_tefflim', 'st_rad', 'st_raderr1', 'st_raderr2', 'st_radlim', 'st_mass', 'st_masserr1', 'st_masserr2', 'st_masslim', 'st_met', 'st_meterr1', 'st_meterr2', 'st_metlim', 'st_metratio', 'st_logg', 'st_logg

# Check stellar parameter consistency for each system

In [8]:
npl = np.array(data["sy_pnum"], dtype="int")

starname = np.array(data["hostname"])
detmet = np.array(data["discoverymethod"])
Mstar = np.array(data["st_mass"])
Rstar = np.array(data["st_rad"])

uniquesys = np.unique(starname)

In [9]:
# some planets have no stellar mass/radius given; others have a value for only a single planet in a system
# for multis where one planet has a stellar mass/radius value, broadcast this to all planets in the system
for i, s in enumerate(uniquesys):
    use = starname == s
    
    # first fix stellar masses
    if np.any(Mstar[use] == ""):
        unique_ms = np.unique(Mstar[use])
        
        if len(unique_ms) == 2:
            Mstar[use] = str(unique_ms[unique_ms != ''].item())
    
    # then fix stellar radii
    if np.any(Rstar[use] == ""):
        unique_rs = np.unique(Rstar[use])
        
        if len(unique_rs) == 2:
            Rstar[use] = str(unique_rs[unique_rs != ''].item())            
            

data["st_mass"] = np.copy(Mstar)
data["st_rad"] = np.copy(Rstar)

In [10]:
RV = data["discoverymethod"] == "Radial Velocity"

npl = np.array(data['sy_pnum'][RV], dtype="int")
Mstar = np.array(data['st_mass'][RV])
Rstar = np.array(data['st_rad'][RV])
starname = np.array(data['hostname'][RV])
planetname = np.array(data['pl_name'][RV])

print("\n\nMissing MASS")
print(np.unique(starname[(Mstar == '')*(npl > 2)]))

print("\n\nMissing RADIUS")
print(np.unique(starname[(Rstar == '')*(npl > 2)]))



Missing MASS
['GJ 163']


Missing RADIUS
['GJ 163' 'GJ 180' 'GJ 433' 'GJ 667 C' 'HD 133131 A' 'HD 133131 B'
 'HD 136352' 'HD 141399' 'HD 160691' 'HD 20781' 'HD 20794' 'HD 27894'
 'HD 31527' 'HD 37124' 'HD 40307' 'HD 69830' 'tau Cet']


# Read in Kepler names

In [11]:
kepnamepath = MAINPATH + "catalogues/kepler_names.txt"

# read in the stellar output parameters
with open(kepnamepath, "r") as infile:
    raw_kepnames = []
    
    for i, line in enumerate(infile):
        raw_kepnames.append(line.split(","))
            
raw_kepnames = np.array(raw_kepnames)

# strip off trailing \newline commands
for i in range(len(raw_kepnames)):
    raw_kepnames[i,-1] = raw_kepnames[i,-1].strip("\n").strip("\ ")

In [12]:
kepnames = {}

for i, k in enumerate(raw_kepnames[0]):
    kepnames[k] = raw_kepnames[1:,i]

# Read in Gaia DR2

In [13]:
gaiapath = MAINPATH + "catalogues/berger_2020_gaia_kepler_tab2_output.txt"

# read in the stellar output parameters
with open(gaiapath, "r") as infile:
    raw_gaia_data = []
    
    for i, line in enumerate(infile):
        raw_gaia_data.append(line.split("&"))
            
raw_gaia_data = np.array(raw_gaia_data)


# strip off trailing \newline commands
for i in range(len(raw_gaia_data)):
    raw_gaia_data[i,-1] = raw_gaia_data[i,-1].strip("\n").strip("\ ")
    
    
gaia_stars = {}

for i, k in enumerate(raw_gaia_data[0]):
    gaia_stars[k] = raw_gaia_data[1:,i]

In [14]:
gaiapath = MAINPATH + "catalogues/berger_2020_gaia_kepler_planets.txt"

gaia_planets = {}

gaia_planets["KIC"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=0)
gaia_planets["radius"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=3)
gaia_planets["radius_err1"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=4)
gaia_planets["radius_err2"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=5)
gaia_planets["sma"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=6)
gaia_planets["sma_err1"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=7)
gaia_planets["sma_err2"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=8)

# Cross-match Kepler vs. Gaia and combine

In [15]:
gaia_kic = np.asarray(gaia_stars["KIC"], dtype="int")

for i in range(len(data[k0])):
    hostname = data["hostname"][i]
    
    if hostname[:3] == "Kep":
        
        for j, kname in enumerate(kepnames["kepler_name"]):
            if kname[:-2] == hostname:
                kic = int(kepnames["kepid"][j])
        
        match = gaia_kic == kic
        
        if np.sum(match) == 1:
            data["st_refname"][i] = "Berger et al. 2020"
            
            data["st_teff"][i] = gaia_stars["iso_teff"][match][0]
            data["st_tefferr1"][i] = gaia_stars["iso_teff_err1"][match][0]
            data["st_tefferr1"][i] = gaia_stars["iso_teff_err2"][match][0]
            data["st_tefflim"][i]  = "0"
            
            data["st_rad"][i] = gaia_stars["iso_rad"][match][0]
            data["st_raderr1"][i] = gaia_stars["iso_rad_err1"][match][0]
            data["st_raderr1"][i] = gaia_stars["iso_rad_err2"][match][0]
            data["st_radlim"][i]  = "0"
            
            data["st_mass"][i] = gaia_stars["iso_mass"][match][0]
            data["st_masserr1"][i] = gaia_stars["iso_mass_err1"][match][0]
            data["st_masserr1"][i] = gaia_stars["iso_mass_err2"][match][0]
            data["st_masslim"][i]  = "0"
            
            data["st_met"][i] = gaia_stars["iso_feh"][match][0]
            data["st_meterr1"][i] = gaia_stars["iso_feh_err1"][match][0]
            data["st_meterr1"][i] = gaia_stars["iso_feh_err2"][match][0]
            data["st_metlim"][i]  = "0"
            data["st_metratio"][i]  = "[Fe/H]"
            
            data["st_logg"][i] = gaia_stars["iso_logg"][match][0]
            data["st_loggerr1"][i] = gaia_stars["iso_logg_err1"][match][0]
            data["st_loggerr1"][i] = gaia_stars["iso_logg_err2"][match][0]
            data["st_logglim"][i]  = "0"

In [16]:
gaia_kic = np.asarray(gaia_planets["KIC"], dtype="int")

count = 0

for i in range(len(data[k0])):
    hostname = data["hostname"][i]
    
    if hostname[:3] == "Kep":
        
        for j, kname in enumerate(kepnames["kepler_name"]):
            if kname[:-2] == hostname:
                kic = int(kepnames["kepid"][j])
        
        match = gaia_kic == kic
        
        if np.sum(match) == 1:
            data["pl_rade"][i] = gaia_planets["radius"][match][0]
            data["pl_radeerr1"][i] = gaia_planets["radius_err1"][match][0]
            data["pl_radeerr1"][i] = gaia_planets["radius_err2"][match][0]
            data["pl_radelim"][i]  = "0"
            
            data["pl_radj"][i] = str(np.round(float(gaia_planets["radius"][match][0])*RERJ,3))
            data["pl_radjerr1"][i] = str(np.round(float(gaia_planets["radius_err1"][match][0])*RERJ,3))
            data["pl_radjerr1"][i] = str(np.round(float(gaia_planets["radius_err2"][match][0])*RERJ,3))
            data["pl_radjlim"][i]  = "0"
            
            
            data["pl_orbsmax"][i] = gaia_planets["sma"][match][0]
            data["pl_orbsmaxerr1"][i] = gaia_planets["sma_err1"][match][0]
            data["pl_orbsmaxerr1"][i] = gaia_planets["sma_err2"][match][0]
            data["pl_orbsmaxlim"][i]  = "0"

# Write out catalogue

In [17]:
WRITENEW = True
if WRITENEW:
    filepath = MAINPATH + 'catalogues/oviraptor_crossmatch_catalog.csv'

    with open(filepath, "w") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(data.keys())
        writer.writerows(zip(*data.values()))